In [1]:
!nvidia-smi

Wed Jan 28 21:29:14 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   47C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip install -U "transformers>=4.41" "accelerate" "datasets" "trl" "peft" "bitsandbytes" "scikit-learn"


In [ ]:
from huggingface_hub import login

# TEMPORARY - DELETE THIS SCRIPT AFTER USE
login(token="")

# Test access
from huggingface_hub import model_info

try:
    info = model_info("meta-llama/Llama-3.2-1B-Instruct")
    print("✓ Access granted!")
except Exception as e:
    print(f"✗ Access denied: {e}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


✓ Access granted!


In [4]:
from sklearn.datasets import fetch_20newsgroups
from datasets import Dataset
import random

random.seed(42)

# Pick a small subset of categories to keep it simple
categories = [
    "sci.space",
    "rec.sport.baseball",
    "talk.politics.misc",
    "comp.graphics",
]

train = fetch_20newsgroups(
    subset="train", categories=categories, remove=("headers", "footers", "quotes")
)
test = fetch_20newsgroups(
    subset="test", categories=categories, remove=("headers", "footers", "quotes")
)

label_names = [
    train.target_names[i] for i in range(len(train.target_names))
]  # same order


def make_rows(data, max_examples=800):
    idxs = list(range(len(data.data)))
    random.shuffle(idxs)
    idxs = idxs[:max_examples]
    rows = []
    for i in idxs:
        text = data.data[i].strip().replace("\n", " ")
        # keep examples short-ish to fit context
        text = text[:1200]

        label = data.target_names[data.target[i]]
        prompt = (
            "You are a classifier.\n"
            f"Choose exactly one label from: {', '.join(categories)}.\n\n"
            f"Text:\n{text}\n\n"
            "Answer with only the label."
        )
        rows.append({"prompt": prompt, "response": label})
    return rows


train_rows = make_rows(train, max_examples=800)  # small training set
test_rows = make_rows(test, max_examples=200)  # small evaluation set

train_ds = Dataset.from_list(train_rows)
test_ds = Dataset.from_list(test_rows)

train_ds[0], len(train_ds), len(test_ds)

({'prompt': 'You are a classifier.\nChoose exactly one label from: sci.space, rec.sport.baseball, talk.politics.misc, comp.graphics.\n\nText:\n=== I aint talking the large or even the "mining companies" I am talking the small miners, the people who have themselves and a few employees (if at all).The people who go out every year and set up thier sluice box, and such and do mining the semi-old fashion way.. (okay they use modern methods toa point).  I am talking the guy who coem to Nome evry year, sets up his tent on the beach (the beach was washed away last year) and sets up his/her sluice box and goes at it "mining". I know the large corps, such as Alaska Gold Company, might complain to..  My opinions are what I learn at the local BS table..  My original thing/idea was that the way to get space mining was to allow the eco-freaks thier way.. As they have done with other mineral development. You can\'t in many places can\'t go to the bathroom in the woods without some form of regulation 

In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "meta-llama/Llama-3.2-1B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=(
        torch.bfloat16 if torch.cuda.is_available() else torch.float32
    ),
)

tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=bnb_config,
)

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/146 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [6]:
def to_chat_text(example):
    messages = [
        {"role": "user", "content": example["prompt"]},
        {"role": "assistant", "content": example["response"]},
    ]
    example["text"] = tokenizer.apply_chat_template(messages, tokenize=False)
    return example


train_ds2 = train_ds.map(to_chat_text)
test_ds2 = test_ds.map(to_chat_text)
train_ds2[0]["text"][:300]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 28 Jan 2026\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nYou are a classifier.\nChoose exactly one label from: sci.space, rec.sport.baseball, talk.politics.misc, comp.graphics.\n\nText'

In [7]:
from peft import LoraConfig

peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
)

In [18]:
from trl import SFTTrainer, SFTConfig

train_ds_only_text = train_ds2.remove_columns(
    [c for c in train_ds2.column_names if c != "text"]
)


training_args = SFTConfig(
    output_dir="llama_20news_lora",
    num_train_epochs=1,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,  # effective batch size = 16
    learning_rate=2e-4,
    logging_steps=10,
    save_strategy="epoch",
    max_length=512,
    fp16=not torch.cuda.is_bf16_supported(),  # use fp16 if bf16 not supported
    bf16=torch.cuda.is_bf16_supported(),
    gradient_checkpointing=True,
    report_to=[],
    dataset_text_field="text",
)

trainer = SFTTrainer(
    model=model,
    processing_class=tokenizer,
    args=training_args,
    train_dataset=train_ds_only_text,
    peft_config=peft_config,
)

trainer.train()
trainer.save_model("llama_20news_lora")

/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:72: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:285: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Adding EOS to train dataset:   0%|          | 0/800 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/800 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/800 [00:00<?, ? examples/s]

Step,Training Loss
10,2.948571
20,2.365052
30,2.195619
40,2.299173
50,2.291632


In [19]:
import re
from tqdm import tqdm


def predict_label(prompt, max_new_tokens=10):
    messages = [{"role": "user", "content": prompt}]
    chat = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    inputs = tokenizer(chat, return_tensors="pt").to(model.device)

    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,  # greedy
            eos_token_id=tokenizer.eos_token_id,
        )

    text = tokenizer.decode(out[0], skip_special_tokens=True)
    # get last line-ish
    pred = text.strip().split("\n")[-1].strip()
    return pred


def normalize(x):
    x = x.strip()
    # sometimes the model may output extra punctuation
    x = re.sub(r"[^a-zA-Z\.\_]", "", x)
    return x


correct = 0
N = 50  # keep it small for speed
for i in tqdm(range(N)):
    prompt = test_rows[i]["prompt"]
    gold = test_rows[i]["response"]
    pred = predict_label(prompt)
    if normalize(pred) == normalize(gold):
        correct += 1

print(f"Accuracy@{N}: {correct/N:.2f} ({correct}/{N})")

  0%|          | 0/50 [00:00<?, ?it/s]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
100%|██████████| 50/50 [00:49<00:00,  1.02it/s]

Accuracy@50: 0.00 (0/50)


In [21]:
def ask(text):
    prompt = (
        "Choose exactly one label from: sci.space, rec.sport.baseball, talk.politics.misc, comp.graphics.\n\n"
        f"Text:\n{text}\n\n"
        "Answer with only the label."
    )
    return predict_label(prompt)


print(ask("NASA announced a new launch window for a satellite mission next month."))
print(ask("The pitcher threw a no-hitter and the fans went wild in the stadium."))
print(ask("The new policy proposal is causing debate across different parties."))
print(ask("Rendering performance improved after switching to a new shading technique."))

sciimportimportimportimportimportimportimportimportimport
recimportimportimportimportimportimportimportimportimport
talk        returnimportimportimportimportimportimportimport
compimportimportimportimportimportimportimportimportimport


In [22]:
categories = [
    "sci.space",
    "rec.sport.baseball",
    "talk.politics.misc",
    "comp.graphics",
]

In [23]:
print(categories)
assert categories == [
    "sci.space",
    "rec.sport.baseball",
    "talk.politics.misc",
    "comp.graphics",
]

['sci.space', 'rec.sport.baseball', 'talk.politics.misc', 'comp.graphics']


In [24]:
print(train_rows[0]["prompt"][:500])
print("----")
print(train_rows[0]["response"])

You are a classifier.
Choose exactly one label from: sci.space, rec.sport.baseball, talk.politics.misc, comp.graphics.

Text:
=== I aint talking the large or even the "mining companies" I am talking the small miners, the people who have themselves and a few employees (if at all).The people who go out every year and set up thier sluice box, and such and do mining the semi-old fashion way.. (okay they use modern methods toa point).  I am talking the guy who coem to Nome evry year, sets up his tent
----
sci.space
